# A Language-independent and Compositional Model for Personality Trait Recognition from Short Texts
## 短文からの性格特性認識のための非言語依存構造的モデル
Fei Liuら2016

## 0. あぶすと
テキストからの著者の自動性格特性認識は多くの試みがなされてきた  
その典型例は、言語的特徴と線形回帰やSVMといった従来の機械学習モデルを組み込んだ手法  
本論では、テキストの原子的特徴-文字-によるでーぷらーにんぐモデルを用い、特性推定のために階層的かつベクトル的な単語・文表現を構築する手法を提案する  
ツイートコーパスでは、この手法は5つの特性と3つの言語に渡り、筆者プロファイリングにおける先行研究と比較して最高水準の結果を残した  
予備的に視覚化された実験による結果は、複雑な人の特性を検出するに足る能力があることを示している  

## 1. いんとろ
めっちゃ自然言語処理ででーぷらーにんぐ使うようになってんねん  
(言語の使用から個人を自動的にモデリングすることは、性別や性格などの社会人口学的特徴が言語の使用に影響を与えるという理解に基づいている  
特に性格特性の研究は、一般的に時間的に安定しているため、信頼できると考えられる  
そのため、目標である筆者をモデル化する手法は、時間の経過につれデータを収集することで充実させている)  
性格認識のための対象となる文、またその応用はここ10年で急速に増大してきた  
しかし、これらの言語的特徴のアプローチは、めっちゃ特徴選択の努力が必要となる  
その上、言語的手法は言語依存であるため、多言語の用途へのモデルの適応が困難である  
別に懸念すべきは、言語使用に関する特性のように、これらの特徴も同様に安定であるという共通した認識である(同じ言語的特徴は、いつでも同じ特性を示す)  
しかしながら、言語と性格間の関係はすべての(多言語の？)コミュニケーションの形に渡り一貫しているわけではなく、より複雑である  
  
これらの課題に対処するため、言語に依存した特徴を作成する必要なしにモデルを様々な言語で動作させることを可能とする、新しい特徴選択なしディープラーニングベースの手法を提案する  
筆者らはこの問題を、テキストの共通原子的な表現(階層的な文字と単語のレベルで)のみを取り出す、教師あり順序回帰問題として捉える  
短文に焦点をあてる  
短文に含まれるノイズ的特徴のため、そのような(短文)テキストの分類は困難であると指摘されている  
なんで、本研究では特徴選択なしに、一度学習すれば5つの特性および3つの言語における性格を推定することができる新しい階層的ニューラルネットワーク構造を提案することにより、この問題に取り組む  
  
本論文では、2つの実験セットを報告する  
1つ目は現在の最先端のモデルと比較した性格推定モデルの有効性の実証  
2つ目は他の特徴選択なしモデルに対する分析  


## 2. 関連研究
余力があったらよむ  

## 3. モデル
まず、現在一般的に用いられている手法の問題と限界を示す  
### 3.1 現在の手法の問題
ディープラーニングモデルを自然言語処理に適用する際、単語ルックアップテーブルを用いて単語をより低次元空間内の密な実数値ベクトルに写像する、といった手法が基本的に用いられる  
ふつうは、この手法がうまくいかせるために大規模なコーパス上で教師なし学習を行い、(たとえばword2vecやGloveなどを使用して)わかりやすい埋め込みセットを取得する  
この手法は構文・意味情報を捉える強い能力を示しており、多くのNLPタスクにうまく適用されているが、Lingによって証明されるように、実際には2つの問題が存在する  

- 一つは、言語が可変であることを前提として、教師なし訓練コーパスの大きさに関係なく、必ず未知語が発声することとなる  
この問題は、TwitterやFacebookといったソーシャルメディアのようなところからのユーザが生成したテキストを取り扱う場合、そこには誤字・その場限りの頭文字語/略語・音声置換語・無意味な文字列といったノイズが多量に含まれるため、さらに顕著になる  
一つの単純な解法としては、すべての未知語を特別なUNKベクトルとして表現するという方法である  
しかしながら、これは重要であるかもしれない未知語の意味を失わせてしまう  
さらに、たとえば"美しい"と"化"から構成される語が確認されたとしても、"美化"という単語に一般化することもできない  
  
- 二つ目は、学習するモデルの多くのパラメータが、過学習を起こしてしまう可能性があるということである  
d次元のベクトルで各単語を表現し、語彙サイズが$|V|$のときに単語ルックアップテーブルのサイズが$d\times |V|$であると仮定すると、通常は数百から数千のオーダーとなる  
繰り返しいうと、これらの問題はノイズの多い領域で特に深刻である  

筆者プロファイルの分野では多数の文字ベースの特徴が研究され、これは文字フラッディング、文字n-gram、感情といった特性推定に有効であることが示されている  
これが筆者らが提案するモデルを動機づけるものであり、文字、単語、また文字レベルにまで深く埋め込まれた性格の機微な表れを活用することができる  

### 3.2 性格特性のためのC2W2S(Char to Word to Sentence)
3.1節で提示した問題について、各単語の構成文字を文字レベル双方向RNNの入力とすることで単語表現を構築する、文字から単語への構成モデル(C2W)を拡張することで対処する  
文は、単語レベルで動作する別の双方向RNN(Word-Bi-RNN)を介して、順方向単語RNNと逆方向単語RNNそれぞれの、最後の隠れ状態と最初の隠れ状態との連結によって表現される  
最終的に、順伝播(フィードフォワード)ニューラルネットワークは入力された文表現を基に特定個人の性格特性のスカラーを予測する  
このモデルの階層的な性質から、これをC2W2S4PT(Char to Word to Sentence for Personality Traits)と名付ける  
具体的な定義は以下のように与えられる。まず、文$s$を単語列${w_1,w_2, ... ,w_i, ... w_m}$とみなし、単語$w_i$はそれぞれ埋め込みを表す文字列$c_{ij}$であると仮定する  
次に、単語$w_i$の表現を構築するため、Char-Bi-RNNは埋め込み文字列$\{c_{i,1}, ... c_{i,n}\}$(ここで、単語$w_i$はn文字で構成されているとする)を入力として受け取り、結果として単語埋め込み$e_{wi}$を得る  
ここで、Bi-RNNの反復素子として、LSTMに匹敵する結果を残し、なおかつ計算量がより少ないという最近の研究から、GRUが採用されている  
具体的には、文字埋め込みはChar-Bi-RNNによって次のように処理される  

$$\vec{z^c_{i,j}} = \sigma( \vec{W^c_z}c_{i,j} + \vec{U^c_{hz}}\vec{h^c_{i,j-1}} + \vec{b^c_z} )$$

$$\vec{r^c_{i,j}} = \sigma( \vec{W^c_r}c_{i,j} + \vec{U^c_{hr}}\vec{h^c_{i,j-1}} + \vec{b^c_r} )$$

$$\vec{\tilde{h}^c_{i,j}} = f(\vec{W^c_z}c_{i,j} + \vec{r^c_{i,j}} \odot \vec{U^c_{hh}}\vec{h^c_{i,j-1}} + \vec{b^c_h} )$$

$$\vec{h^c_{i,j}} = \vec{z^c_{i,j}} \odot \vec{h^c_{i,j-1}} + (1 - \vec{z^c_{i,j}}) \odot \vec{\tilde{h}^c_{i,j}}$$

ここで、$\odot$は要素単位の積であり、$\sigma$はシグモイド関数、fは双曲線正接関数tanhを表す  
さらに、$ \vec{W^c_z},\vec{W^c_r},\vec{W^c_h},\vec{U^c_{hz}},\vec{W^c_{hr}},\vec{U^c_{hh}} $は学習するパラメータ行列であり、$\vec{b^c_z},\vec{b^c_r},\vec{b^c_h}$はバイアス(しきい値)である  
さらに、Char-Bi-RNNは順方向に加え、また別の重み行列とバイアス項を持ったGRUで逆方向から文字列を処理する(これを$\overleftarrow{h^c_{i,j}}$と表す)  
同じ文字の埋め込みは順方向と逆方向で共有されることに注意する  
最終的に、単語$w_i$をChar-RNNの順方向の最後と逆方向の最初の隠れ層の連結として表現する  

$$e_{wi} = \begin{bmatrix} \vec{h^c_{i,j}} \\ \overleftarrow{h^c_{i,j}} \end{bmatrix}$$  

文表現は単語表現と同様の方法で構築される  
単語レベルで動作する別の双方向RNNにより、$e_{w,i}$(ここで、$i\in [1,n]$であり、すべての単語表現はそれらの構成文字から成り立つ)が処理される  

$$\vec{z^w_{i}} = \sigma( \vec{W^w_z}e_{wi} + \vec{U^w_{hz}}\vec{h^w_{i-1}} + \vec{b^w_z} )$$

$$\vec{r^w_{i}} = \sigma( \vec{W^w_r}e_{wi} + \vec{U^w_{hr}}\vec{h^w_{i-1}} + \vec{b^w_r} )$$

$$\vec{\tilde{h}^w_i} = f(\vec{W^w_h}e_{wi} + \vec{r^w_i} \odot \vec{U^w_{hh}}\vec{h^w_{i-1}} + \vec{b^w_h} )$$

$$\vec{h^w_i} = \vec{z^w_i} \odot \vec{h^w_{i-1}} + (1 - \vec{
z^w_i}) \odot \vec{\tilde{h}^w_i}$$

WもUもbもさっきと同じ重みとバイアス  
最終的に文sをWord=RNNの順方向の最後と逆方向の先頭の連結として表現する(ここで文sはm語から成るとする)  

$$e_{s} = \begin{bmatrix} \vec{h^w_m} \\ \overleftarrow{h^w_1} \end{bmatrix}$$  

最後に、特定人の性格特性スコアはMLP(多層パーセプトロン)で推定され、埋め込まれた文を入力として推定スコア$\hat{y}_s$を返す  

$$\mathbf{h}_s = ReLu(\mathbf{W}_{eh}e_s + b_h)$$

$$\hat{y}_s = \mathbf{W}_{hy}\mathbf{h}_s+b_y$$

ここで、ReLU(Rectified Linear Unit)はReLU(x) = max(0,x)で定義され、$\mathbf{W}_{eh},\mathbf{W}_{hy}$は学習するパラメータ(重み)行列でbはバイアスである  
$\mathbf{h}_s$はMLPの隠れ表現  

すべての訓練対象となるパラメータ・埋め込み行列およびバイアス項は、目的関数として平均二乗和誤差を用いて一律に最適化される  

$$L(\theta) = \frac{1}{n} \sum^n_{i=1} (y_{si}-\hat{y}_{si})^2$$

ここで$y_{si}$は文$s_i$の信頼できる性格特性スコアであり、$\theta$は学習するモデルのすべてのパラメータ、埋め込み行列とバイアス項の集合である  
ここで提案されたモデルには言語依存の成分が含まれていないことに注意  


## 4. 実験と結果
提案モデルの有効性を確かめるため、特徴選択あり/なしのモデルと比較して二つの実験設定の上で評価を行った  
前者では、ユーザレベルで今回提案したモデル(特徴選択なし言語非依存モデル)と、言語的特徴を用いるこの時点においての最先端のモデルを比較する  
一方後者では、他の短文における特徴選択なしモデルとの性能差を調査した  
どちらの設定においても、英語とスペイン語ではよりよい結果を残し、イタリア語でも同等の結果を示した  

### 4.1 データセットと前処理
本実験ではRangelらの筆者プロファイリングデータセットより英語・スペイン語・イタリア語のデータセットを用いた  
内容はツイッターより収集された14166の英文ツイート、9879の西文(スペイン)ツイート、3687の伊文ツイートである  
(それぞれ、152人・110人・38人のユーザからなる)  
データの制限上ドイツ語は含まない  
それぞれのユーザは各自のツイート(平均100)と信頼できる性格ラベル(スコアは-0.5から0.5で表現される)と関連付けられている  
このスコアは小規模のBig5テストであるBFI-10(現時点で最もしっかりとした、性格認識のため広く受け入れられているテスト)による自己評価結果に従い計算した  
実験では、各ツイートをTwokenizerを用いてトークン化し、ユーザの返信やハッシュタグ付きのトピックを保持している  
ユーザの返信やURLは、ツイートに用いられる大半の言葉とは違い、その対象となる先に向けられたものであるため、字面上にはほとんど意味がないと思われる  h
したがって、これらの特徴を一文字に正規化する(@hogehoge→@、http://hogehoge → ^)ことで、モデル時に不必要な文字が筆者の性格に直接影響を与えてしまうリスクを抑える  

### 4.2 評価指標
テストコーパスはPAN2015主催者により使用が差し控えられているため、代わりにk-分割(k=5,10)交差検証を行い利用可能なデータセットに対する性能評価を比較する  
性能の評価を行うにあたり、誤差二乗平均平方根(RMSE)を用い、タスクに応じてツイートまたはユーザのRMSE($RMSE_{tweet},RMSE_{user}$)を測定した  

$$ RMSE_{tweet} = \sqrt{ \frac{ \sum^T_{i=1} ( y_{s_i}-\hat{ y_{s_i} } )^2 }{T} } $$

$$ RMSE_{user}= \sqrt{\frac{\sum^U_{i=1}( y_{user_i}-\hat{ y_{user_i}})^2}{U}} $$

ここで$y_{s_i}$と$\hat{y_{s_i}}$はそれぞれコーパスに含まれるi番目のツイートに対するgold standard値と推測値を表し、$y_{user_i}$,$\hat{y_{user_i}}$はユーザーレベルの値である  
T、Uはそれぞれコーパスに含まれる総ツイート数と総ユーザー数である  
この実験で用いられるデータセットは、各ユーザにそれぞれただ一つの性格特性が割り当てられ、同じアカウントから収集されたツイートはその筆者と同じ性格特性の割当を受け継ぐことに注意する  
ユーザレベルの予測特性値$\hat{y_{user_i}}$は次のように計算される

$$\hat{y_{user_i}}=\frac{1}{T_i}\sum^{T_i}_{j=1}\hat{y_{s_i}}$$

ここで、$T_i$は$user_i$に含まれる総ツイート数である  
以下で行う実験ではこれら二つのRMSEを結果とした計測する  

### 4.3 最先端モデルに対する評価
実験では、提案モデルは追加特徴のないテキストのみに依って性格特性値を予測するよう訓練されることに注意する  
比較を可能とするため、心理学的研究に基づく言語的特徴を組み込んだ現在の最先端モデルを用いて、ユーザレベルでC2W2S4PTを評価する  
5分割交差検証と10分割交差検証では違うモデルを比較対象とする  
これらのテキストレベルでのスコアを予測し、各ユーザについて平均を計算した作業に加え、Mirkinがいうように単一ユーザのつぶやきを連結した追加作業を含む  
また、ベースラインにはすべてのスコアの平均を各ユーザに割り当てる単純な方法を採択する  
C2W2S4PTの訓練にはAdamを用い、バッチサイズを32・エポック数を100にして次のハイパーパラメータを定めた  
$$\vec{h^c_{i,j}},\overleftarrow{h^c_i,j}\in \mathbb{R}^{256}$$  
$$E_c\in\mathbb{R}^{50\times |C|}$$ エンベッディング出力のドロップアウトレート: 0.5(?)  
$$\vec{h^w_i},\overleftarrow{h^w_i}\in \mathbb{R}^{256}$$  
$$\mathbf{W_{hy}\in\mathbb{R}^{256\times 1}}, b_y\in\mathbb{R}$$
$$\mathbf{W_{eh}\in\mathbb{R}^{512\times 256}, \mathbf{b}\in\mathbb{R}^{256}}$$

5因子モデルの特性であるEXT,STA,AGR,CON,OPNについて、$RMSE_{user}$の結果を示す  

#### C2W2S4PTモデルは現在の最先端手法よりも良い結果を示した(EN, ES)
5分割交差検証のグループでは、提案手法はESにおけるCONを除きベースラインと比較してよりよい性能を達成している   
10分割交差検証では、ES,ENのすべての人格特性においてベースラインである2手法を圧倒し、優位性を示した  
5,10分割交差検証のいずれのグループでも、提案モデルは(特に10分割交差検証において顕著に)人手の特徴による助けなしにベースライン手法を上回った  

#### 5分割交差検証におけるCON in ES
Suleaの手法が驚くほどに高い性能を示したことについて、筆者は過適合による可能性が高いと考える?  

#### ITにおいて優位性が明確でない
これは、英語とスペイン語のデータセットがそれぞれ14k/10kツイートあるのに対しイタリア語は4kしかツイートがないために、提案モデルが適切に学習することが制限されている可能性があるためである  

### 4.4 他の特徴選択なし手法との比較評価
性格特性の推定ではユーザレベルで評価するのが一般的だけど、モデルの性能を細かく調査するためツイートレベルでの結果も調べてみる  
評価をおこなうにあたり、公正な評価をするためにテキストの表面情報のみを組み込んだベースラインをいろいろ作成した  
まず、4.3節で示したユーザ平均のベースライン  
次に二つのBoWに基づくシステムとしてRandom ForestとSVMを採用した  
また、これらの従来の機械学習ベースのモデルに加え、文字と単語レベルで機能する二つの単純なRNNベースモデルであるBi-GRU-CharとBi-GRU-Wordをさらに実装した  
これらのモデルについて、ハイパーパラメータの微調整はおこなわれなかった  
以上の手法それぞれについて10分割交差検証を行った結果を示す  

#### 提案モデルはSVMやRFと同程度の性能を示した
提案モデルはENにおけるAGRとESにおけるSTAを除いて、ほぼすべての特性において最も良い結果を残した  
ハイパーパラメータを調整していないSVMとRFに対し、提案モデルが特徴選択なく同等の性能を示したことは注目すべき点である  

#### 提案モデルはENとESにおいてRNNに基づくベースラインよりも良い性能を達成した 
Bi-GRU-Wordと比較すると、提案モデルは学習するパラメータが少ないためにオーバーフィッティングが起きにくいが、Bi-GRU-Wordは大量の単語埋め込み行列を必要とする  
Bi-GRU-Charに関しては、テキストを文字列として表現することから起こりえる過剰な長さ(の文)による情報損失なしに、提案モデルが慣用句を処理することに成功が起因していると考えられる  

#### ITにおいて、提案モデルの性能がBGWを下回った
Bi-GRU-Wordは、すべての性格特性にわたって最良の結果を達成し、提案モデルと比較すると内2つは上回り、3つは同位であった  
4.3節で述べたように、十分な量のイタリア語のデータが性能の変動を起こすか、ということを別にして、Bi-GRU-Wordが良い結果を残したことについて解析するため、さらなる調査が必要である  

### 4.5 見える化
提案モデルによる特徴の自動学習を調査するため、性格特性の一つであるEXTについて、文におけるその主成分分析を二次元の散布図にプロットした  
例として、乱択によって100件のツイートを選ぶ  
Extraversion(外向性)は最も一般に研究され、よく知られている特性であるため、これを可視化に用いる  
提案モデルを用いて自動構築された文表現を図2に示す  
ここで、正のEXTと負のEXTという二つのクラスタが容易に識別できており、これらは互いに交差していることがわかる  
以下に、図２内で強調された３つの例文を示す  
 - POS7 : "@user: Feeling like you're not good enough is probably the worst thing to feel."(満たされないと思うことが、たぶん最も精神に悪い)
 - NEG3 : "Being god ain't enough lately."(最近いいことが全然ねえ)
 - POS20: "o.O Lovely"(o.O くそかわ)
 
POS7とNEG3の二つの例は、意味上では本質的に類似しているが、分布距離では互いに遠くプロットされている  
意味が類似しているにも関わらず、これらの例が所有する言語属性は、一般的に異なるEXTが関連付けられると考えられている  
POS7のようなより長いツイートと第二人称代名詞の使用は、筆者がより他者を内包していると思われるのに対し、NEG3は自己完結的で短いため、より内向的な要素を含んでいることを表している  
３つめの例であるPOS20に関しては、内向的な空間に含まれているかのように見えるが、実際は外向なツイートである  
(ツイートが)短いこととは別に、POS20には回転していない東洋形式の感情表現である(o.O)が使われており、これはソーシャルメディアでは内向と結びつけられる側面がある  
このことに関しては、Ambivertらによってそのへんの柔軟性を明らかにしたモデルの説明があったのだけれども、まああんまり考えなくていいんじゃない  
でもこのモデルが特徴選択なしに言語の次元を捉えられることは留意しといたほうがいいかもね  
